# **DSFM Exercise**: Open-source Models - style transfer (SOLUTIONS)

Creator: [Data Science for Managers - EPFL Program](https://www.dsfm.ch)  
Source:  [https://github.com/dsfm-org/code-bank.git](https://github.com/dsfm-org/code-bank.git)  
License: [MIT License](https://opensource.org/licenses/MIT). See open source [license](LICENSE) in the Code Bank repository. 

-------------

## Overview

In this exercise, we leverage an open-source model to show the power of re-using existing work from the data science community. We will convert the style of an image, based on a pre-trained open-source model.

This *neural style transfer* takes a *content image* and a *style reference image* (e.g. by Picasso, Kandinsky, Van Gogh). The goal is to "paint" the content image in the style of the reference image, using neural networks.

Original paper: *A Neural Algorithm of Artistic Style* by [Gatys et al. (2015)](https://arxiv.org/abs/1508.06576)

---------

## Part 0: Setup

In [ ]:
# Imports
import os
import tensorflow as tf
import tensorflow_hub as hub
import IPython.display as display
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import PIL.Image
import time
import functools

# Define plotting format
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

In [ ]:
# Helper functions
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

def imshow(image, title=None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
        plt.title(title)

# **MAIN EXERCISE**

## Part 1: Upload image

Upload an image to the directory of this notebook. You can (1) upload an image from your computer or (2) copy an image from the web. We encourage the former – it's more fun.

**Q 1:** Upload image in `.jpg` format and replace the existing `myImage.jpg`.

**Q 2:** Rename image to `myImage.jpg`.

## Part 2: Choose style image

Now comes the creative part. Choose one of the available style reference images below.

**Q 1:** Define the path to the style image in the `/styles` directory. 

Hint: Create a variable called `content_path` to reference the style image you want to use.

In [ ]:
style_path = 'styles/kandinsky.jpg'
# style_path = 'styles/vanGogh.jpg'
# style_path = 'styles/monet.jpg'

**Q 2:** Show the style image.

Hint: Use the `load_img()` and `imshow()` helper functions.

In [ ]:
style_image = load_img(style_path)
imshow(style_image, 'Style Image')


## Part 3: Apply open-source model

We now download an open-source, pre-trained neural network to "paint" our image in the style above. The model is available on the TensorFlow Hub [here](https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2).

In [ ]:
# Download the model
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

**Q 1:** Apply the style to your image.

Hint: Use the `hub_model()` with the `content_image` and `style_image` as inputs.

In [ ]:
content_image = load_img('myImage.jpg')
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]


**Q 2:** Plot the stylized image

In [ ]:
tensor_to_image(stylized_image)
